In [65]:
import datetime
import logging
from collections import defaultdict

import dill
import numpy as np
import pymongo
import pandas as pd
from sklearn.linear_model import LogisticRegression
from typing import Any

from CrossValidation import cross_validation
from Settings import Settings
from cost_functions import *
from crel_helper import get_cr_tags
from function_helpers import get_function_names, get_functions_by_name
from results_procesor import ResultsProcessor, __MICRO_F1__
from searn_parser import SearnModelTemplateFeatures
from template_feature_extractor import *
from window_based_tagger_config import get_config
from wordtagginghelper import merge_dictionaries

import sys
sys.path.append("/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/notebooks/Co-Reference Resolution/Results")

from results_common import get_essays

In [67]:
client = pymongo.MongoClient()
db = client.metrics

# Data Set Partition
CV_FOLDS = 5
MIN_FEAT_FREQ = 5

# Global settings

settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"
# NOTE: These predictions are generated from the "./notebooks/SEARN/Keras 
#  - Train Tagger and Save CV Predictions For Word Tags.ipynb" notebook
# used as inputs to parsing model
coref_root = root_folder + "CoReference/"
coref_output_folder = coref_root + "CRel/"

config = get_config(training_folder)
results_processor = ResultsProcessor(dbname="metrics_coref_causal2")

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [68]:
!pwd

/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/notebooks/Co-Reference Resolution/Causal Relations


In [73]:
import inspect, os
print(inspect.getfile(inspect.currentframe())) # script filename (usually with path)
print(os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))) # script directory

<ipython-input-73-e3e39e85dc45>
/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/notebooks/Co-Reference Resolution/Causal Relations


In [3]:
train_fname = coref_output_folder + "training_crel_anatagged_essays_most_recent_code.dill"
with open(train_fname, "rb") as f:
    pred_tagged_essays_train = dill.load(f)

test_fname = coref_output_folder + "test_crel_anatagged_essays_most_recent_code.dill"
with open(test_fname, "rb") as f:
    pred_tagged_essays_test = dill.load(f)

len(pred_tagged_essays_train),len(pred_tagged_essays_test)

(902, 226)

In [4]:
# doesn't matter with parser for this
stanford_coref_predictions_folder = root_folder + "CoReference/"

orig_pred_tagged_essays_train = get_essays(stanford_coref_predictions_folder, "Training")
orig_pred_tagged_essays_test = get_essays(stanford_coref_predictions_folder, "Test")

# Load original CREL data - without anaphor codes or predictions
# NOTE that these files differ for the SC data
rnn_predictions_folder = root_folder + "Predictions/Bi-LSTM-4-SEARN/"

train_fname = rnn_predictions_folder + "essays_train_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill"
with open(train_fname, "rb") as f:
    orig_pred_tagged_essays_train = dill.load(f)

test_fname = rnn_predictions_folder + "essays_test_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill"
with open(test_fname, "rb") as f:
    orig_pred_tagged_essays_test = dill.load(f)

len(orig_pred_tagged_essays_train), len(orig_pred_tagged_essays_test)

(902, 226)

In [5]:
e = orig_pred_tagged_essays_train[0]
set(e.pred_tagged_sentences[0]) - set(pred_tagged_essays_train[0].pred_tagged_sentences[1])

{'50'}

In [6]:
EMPTY = "Empty"
from BrattEssay import ANAPHORA

def get_different_predicted_tags_by_sent(essays_a, essays_b):
    assert len(essays_a) == len(essays_b)
    diff_by_sent = []
    for ea, eb in zip(essays_a, essays_b):
        assert len(ea.sentences) == len(eb.sentences)
        assert len(ea.pred_tagged_sentences) == len(eb.pred_tagged_sentences)
        for asent, bsent in zip(ea.pred_tagged_sentences, eb.pred_tagged_sentences):
            unique_atags = set(asent)
            if EMPTY in unique_atags:
                unique_atags.remove(EMPTY)
            unique_btags = set(bsent)
            if EMPTY in unique_btags:
                unique_btags.remove(EMPTY)
            diff_by_sent.append(unique_btags - unique_atags)
    return diff_by_sent

def to_is_valid_crel(tags):
    filtered = set()
    for t in tags:
        t_lower = t.lower()
        if "rhetorical" in t_lower or "change" in t_lower or "other" in t_lower:
            continue
        if "->" in t and ANAPHORA not in t:
            filtered.add(t)
    return filtered

def get_different_crel_tags_by_sent(essays_a, essays_b):
    assert len(essays_a) == len(essays_b)
    diff_by_sent = []
    for ea, eb in zip(essays_a, essays_b):
        assert len(ea.sentences) == len(eb.sentences)
        for asent, bsent in zip(ea.sentences, eb.sentences):
            all_atags, all_btags = set(), set()
            for (awd, atags), (bwd, btags) in zip(asent, bsent):
                assert awd == bwd
                all_atags.update(to_is_valid_crel(atags))
                all_btags.update(to_is_valid_crel(btags))
            
            diff_by_sent.append(all_btags - all_atags)
    return diff_by_sent

def get_crel_tags_by_sent(essays_a):
    crels_by_sent = []
    for ea in essays_a:
        for asent in ea.sentences:
            all_atags = set()
            for awd, atags in asent:
                all_atags.update(to_is_valid_crel(atags))
            crels_by_sent.append(all_atags)
    return crels_by_sent

In [7]:
cr_tags = get_cr_tags(train_tagged_essays=pred_tagged_essays_train, tag_essays_test=pred_tagged_essays_test)
# cv_folds  = [(pred_tagged_essays_train, pred_tagged_essays_test)]  # type: List[Tuple[Any,Any]]
cv_folds = cross_validation(pred_tagged_essays_train, CV_FOLDS)

# cv_folds  = [(pred_tagged_essays_train, pred_tagged_essays_test)]  # type: List[Tuple[Any,Any]]

In [8]:
def evaluate_model(
        collection_prefix: str,
        folds: List[Tuple[Any, Any]],
        extractor_fn_names_lst: List[str],
        cost_function_name: str,
        beta: float,
        ngrams: int,
        stemmed: bool,
        max_epochs: int,
        down_sample_rate=1.0) -> float:

    if down_sample_rate < 1.0:
        new_folds = []  # type: List[Tuple[Any, Any]]
        for i, (essays_TD, essays_VD) in enumerate(folds):
            essays_TD = essays_TD[:int(down_sample_rate * len(essays_TD))]
            essays_VD = essays_VD[:int(down_sample_rate * len(essays_VD))]
            new_folds.append((essays_TD, essays_VD))
        folds = new_folds  # type: List[Tuple[Any, Any]]

    serial_results = [
        model_train_predict(essays_TD, essays_VD, extractor_fn_names_lst, cost_function_name, ngrams, stemmed, beta, max_epochs)
        for essays_TD, essays_VD in folds
    ]

    cv_sent_td_ys_by_tag, cv_sent_td_predictions_by_tag = defaultdict(list), defaultdict(list)
    cv_sent_vd_ys_by_tag, cv_sent_vd_predictions_by_tag = defaultdict(list), defaultdict(list)

    # record the number of features in each fold
    number_of_feats = []

    # Parallel is almost 5X faster!!!
    cv_td_preds_by_sent = []
    cv_vd_preds_by_sent = []
    for (num_feats,
         sent_td_ys_bycode, sent_vd_ys_bycode,
         sent_td_pred_ys_bycode, sent_vd_pred_ys_bycode, td_preds_by_sent, vd_preds_by_sent) in serial_results:
        number_of_feats.append(num_feats)

        merge_dictionaries(sent_td_ys_bycode, cv_sent_td_ys_by_tag)
        merge_dictionaries(sent_vd_ys_bycode, cv_sent_vd_ys_by_tag)
        merge_dictionaries(sent_td_pred_ys_bycode, cv_sent_td_predictions_by_tag)
        merge_dictionaries(sent_vd_pred_ys_bycode, cv_sent_vd_predictions_by_tag)
        
        cv_td_preds_by_sent.append(td_preds_by_sent)
        cv_vd_preds_by_sent.append(vd_preds_by_sent)

    # Mongo settings recording
    avg_feats = np.mean(number_of_feats)
    sent_algo = "Shift_Reduce_Parser_LR"

    parameters = dict(config)
    parameters["extractors"] = list(extractor_fn_names_lst)
    parameters["num_extractors"] = len(extractor_fn_names_lst)
    parameters["cost_function"] = cost_function_name
    parameters["beta"] = beta
    parameters["max_epochs"] = max_epochs
    parameters["no_stacking"] = True
    parameters["algorithm"] = str(BASE_LEARNER_FACT())
    parameters["ngrams"] = str(ngrams)
    parameters["num_feats_MEAN"] = avg_feats
    parameters["num_feats_per_fold"] = number_of_feats
    parameters["min_feat_freq"] = MIN_FEAT_FREQ
    parameters["stemmed"] = stemmed

    print("Mean num feats: {avg_feats:.2f}".format(avg_feats=avg_feats))

    TD = collection_prefix + "_TD"
    VD = collection_prefix + "_VD"
    if down_sample_rate < 1.0:
        print("Down sampling at rate: {rate:.5f}, storing temp results".format(rate=down_sample_rate))
        parameters["down_sample"] = down_sample_rate
        CB_SENT_TD, CB_SENT_VD = "__tmp_" + TD, "__tmp_" + TD
    else:
        CB_SENT_TD, CB_SENT_VD = TD, VD

    sent_td_objectid = results_processor.persist_results(CB_SENT_TD, cv_sent_td_ys_by_tag,
                                                         cv_sent_td_predictions_by_tag, parameters, sent_algo)
    sent_vd_objectid = results_processor.persist_results(CB_SENT_VD, cv_sent_vd_ys_by_tag,
                                                         cv_sent_vd_predictions_by_tag, parameters, sent_algo)

    # print(processor.results_to_string(sent_td_objectid, CB_SENT_TD, sent_vd_objectid, CB_SENT_VD, "SENTENCE"))
    metric = results_processor.get_metric(CB_SENT_VD, sent_vd_objectid, __MICRO_F1__)
    return metric, cv_sent_td_ys_by_tag, cv_sent_td_predictions_by_tag, cv_td_preds_by_sent, cv_sent_vd_ys_by_tag, cv_sent_vd_predictions_by_tag, cv_vd_preds_by_sent

In [9]:
def predict_by_sent(tagged_essays, model):
    predict_by_sent = []
    for essay_ix, essay in enumerate(tagged_essays):
        for sent_ix, taggged_sentence in enumerate(essay.sentences):
            predicted_tags = essay.pred_tagged_sentences[sent_ix]
            pred_relations = model.predict_sentence(taggged_sentence, predicted_tags)
            # Store predictions for evaluation
            predict_by_sent.append(pred_relations)
    return predict_by_sent

def model_train_predict(essays_TD, essays_VD, extractor_names, cost_function_name, ngrams, stemmed, beta, max_epochs):
    extractors = get_functions_by_name(extractor_names, all_extractor_fns)
    # get single cost function
    cost_fn = get_functions_by_name([cost_function_name], all_cost_functions)[0]
    assert cost_fn is not None, "Cost function look up failed"
    # Ensure all extractors located
    assert len(extractors) == len(extractor_names), "number of extractor functions does not match the number of names"

    template_feature_extractor = NonLocalTemplateFeatureExtractor(extractors=extractors)
    if stemmed:
        ngram_extractor = NgramExtractorStemmed(max_ngram_len=ngrams)
    else:
        ngram_extractor = NgramExtractor(max_ngram_len=ngrams)
    parse_model = SearnModelTemplateFeatures(feature_extractor=template_feature_extractor,
                                             cost_function=cost_fn,
                                             min_feature_freq=MIN_FEAT_FREQ,
                                             ngram_extractor=ngram_extractor, cr_tags=cr_tags,
                                             base_learner_fact=BASE_LEARNER_FACT,
                                             beta=beta,
                                             # log_fn=lambda s: print(s))
                                             log_fn=lambda s: None)

    parse_model.train(essays_TD, max_epochs=max_epochs)

    num_feats = template_feature_extractor.num_features()

    sent_td_ys_bycode = parse_model.get_label_data(essays_TD)
    sent_vd_ys_bycode = parse_model.get_label_data(essays_VD)

    sent_td_pred_ys_bycode = parse_model.predict(essays_TD)
    sent_vd_pred_ys_bycode = parse_model.predict(essays_VD)

    td_preds_by_sent = predict_by_sent(essays_TD, parse_model)
    vd_preds_by_sent = predict_by_sent(essays_VD, parse_model)
    
    return num_feats, sent_td_ys_bycode, sent_vd_ys_bycode, sent_td_pred_ys_bycode, sent_vd_pred_ys_bycode, td_preds_by_sent, vd_preds_by_sent

In [10]:
LINE_WIDTH = 80

# other settings
DOWN_SAMPLE_RATE = 1.0  # For faster smoke testing the algorithm
BASE_LEARNER_FACT = None
COLLECTION_PREFIX = "CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_MOST_RECENT_CODE"

# some of the other extractors aren't functional if the system isn't able to do a basic parse
# so the base extractors are the MVP for getting to a basic parser, then additional 'meta' parse
# features from all_extractors can be included
base_extractors = [
    single_words,
    word_pairs,
    three_words,
    between_word_features
]

all_extractor_fns = base_extractors + [
    word_distance,
    valency,
    unigrams,
    third_order,
    label_set,
    size_features
]

all_cost_functions = [
    micro_f1_cost,
    micro_f1_cost_squared,
    micro_f1_cost_plusone,
    micro_f1_cost_plusepsilon,
    binary_cost,
    inverse_micro_f1_cost,
    uniform_cost
]

all_extractor_fn_names = get_function_names(all_extractor_fns)
base_extractor_fn_names = get_function_names(base_extractors)
all_cost_fn_names = get_function_names(all_cost_functions)

In [11]:
ngrams = 1
stemmed = True
cost_function_name = micro_f1_cost_plusepsilon.__name__
dual = True
fit_intercept = True
beta = 0.5
max_epochs = 2
C = 0.5
penalty = "l2"

# Train CV

In [12]:
BASE_LEARNER_FACT = lambda : LogisticRegression(dual=dual, C=C, penalty=penalty, fit_intercept=fit_intercept)
best_extractor_names = ['single_words', 'between_word_features', 'label_set',
                                    'three_words', 'third_order', 'unigrams'] # type: List[str]
result = evaluate_model(
    collection_prefix=COLLECTION_PREFIX,
    folds=cv_folds,
    extractor_fn_names_lst=best_extractor_names,
    cost_function_name=cost_function_name,
    ngrams=ngrams,
    beta=beta,
    stemmed=stemmed,
    down_sample_rate=DOWN_SAMPLE_RATE,
    max_epochs=max_epochs)

Mean num feats: 27541.40


In [13]:
metric, cv_sent_td_ys_by_tag, cv_sent_td_predictions_by_tag, cv_td_preds_by_sent, \
    cv_sent_vd_ys_by_tag, cv_sent_vd_predictions_by_tag, cv_vd_preds_by_sent = result

# Compute Anaphora CRel Accuracy

In [14]:
def metrics_to_df(metrics):
    import Rpfa

    rows = []
    for k,val in metrics.items():
        if type(val) == Rpfa.rpfa:
            d = dict(val.__dict__) # convert obj to dict
        elif type(val) == dict:
            d = dict(val)
        else:
            d = dict()
        d["code"] = k
        rows.append(d)
    return pd.DataFrame(rows)

def get_metrics_from_dict(act_ys_bycode, pred_ys_bycode, expected_tags):
    assert len(act_ys_bycode.keys()) == len(pred_ys_bycode.keys()) == len(expected_tags), "Miss-matched codes"
    first_tag = list(expected_tags)[0]
    last_tag  = list(expected_tags)[-1]
    assert len(act_ys_bycode[first_tag]) == len(pred_ys_bycode[first_tag]), "Different numbers of words"
    assert len(act_ys_bycode[last_tag])  == len(pred_ys_bycode[last_tag]),  "Different numbers of words"

    metrics = ResultsProcessor.compute_metrics(act_ys_bycode, pred_ys_bycode)
    mean_metrics = ResultsProcessor.compute_mean_metrics(act_ys_bycode, pred_ys_bycode)
    return metrics_to_df(mean_metrics)

def compute_metrics_from_essays(tagged_esssays, expected_tags):
    act_ys_bycode  = get_wd_level_lbs(  tagged_esssays, expected_tags)
    pred_ys_bycode = get_wd_level_preds(tagged_esssays, expected_tags)
    return get_metrics_from_dict(act_ys_bycode, pred_ys_bycode, expected_tags)

In [15]:
# cv_folds = cross_validation(pred_tagged_essays_train, CV_FOLDS)
orig_cv_folds = cross_validation(orig_pred_tagged_essays_train, CV_FOLDS)

In [16]:

def build_metrics_by_code(td_preds_by_sent, diffs_train, diffs_crel_train, ys_by_code_train, preds_by_code_train):
    for pred_crels, pred_ccodes, crels in zip(td_preds_by_sent, diffs_train, diffs_crel_train):    
        for crel in cr_tags:
            ys_by_code_train[crel].append( 1 if crel in crels else 0)

        # filter predictions to only those that were produced as a result of the ana tags
        ana_preds = set()
        for crel in pred_crels:
            l,r = crel.split("->")
            lcode = l.split(":")[-1].strip()
            rcode = r.split(":")[-1].strip()
            if lcode in pred_ccodes or rcode in pred_ccodes:
                ana_preds.add(crel)

        for crel in cr_tags:
            preds_by_code_train[crel].append( 1 if crel in ana_preds else 0)

In [17]:
# Predicted Concept Codes - correspond to Ana tags
preds_by_code_train = defaultdict(list)
ys_by_code_train    = defaultdict(list)

preds_by_code_test = defaultdict(list)
ys_by_code_test    = defaultdict(list)

for (orig_td, orig_vd), (td, vd), td_preds_by_sent, vd_preds_by_sent in zip(orig_cv_folds, cv_folds, cv_td_preds_by_sent, cv_vd_preds_by_sent):
    
    diffs_train = get_different_predicted_tags_by_sent(essays_a=orig_td, essays_b=td)
    diffs_test  = get_different_predicted_tags_by_sent(essays_a=orig_vd, essays_b=vd)

    # Actual Ana CRels
    diffs_crel_train = get_different_crel_tags_by_sent(essays_a=orig_td, essays_b=td)
    diffs_crel_test  = get_different_crel_tags_by_sent(essays_a=orig_vd, essays_b=vd)
    
    build_metrics_by_code(td_preds_by_sent, diffs_train, diffs_crel_train, ys_by_code_train, preds_by_code_train)
    build_metrics_by_code(vd_preds_by_sent, diffs_test,  diffs_crel_test,  ys_by_code_test,  preds_by_code_test)

## CV Metrics

In [18]:
df_train = get_metrics_from_dict(ys_by_code_train, preds_by_code_train, cr_tags)
df_train[df_train.code == "MICRO_F1"][["accuracy", "f1_score", "precision", "recall"]]

,accuracy,f1_score,precision,recall
95,0.999681,0.062378,0.484848,0.033333


In [19]:
df_test = get_metrics_from_dict(ys_by_code_test, preds_by_code_test, cr_tags)
df_test[df_test.code == "MICRO_F1"][["accuracy", "f1_score", "precision", "recall"]]

,accuracy,f1_score,precision,recall
95,0.999681,0.062257,0.470588,0.033333


# Train for Test Set Eval

In [31]:
orig_test_folds  = [(orig_pred_tagged_essays_train, orig_pred_tagged_essays_test)]  # type: List[Tuple[Any,Any]]
test_folds     = [(pred_tagged_essays_train, pred_tagged_essays_test)]  # type: List[Tuple[Any,Any]]

In [21]:
result_test = evaluate_model(
    collection_prefix=COLLECTION_PREFIX,
    folds=test_folds,
    extractor_fn_names_lst=best_extractor_names,
    cost_function_name=cost_function_name,
    ngrams=ngrams,
    beta=beta,
    stemmed=stemmed,
    down_sample_rate=DOWN_SAMPLE_RATE,
    max_epochs=max_epochs)

Mean num feats: 30804.00


In [26]:
metric, cv_sent_td_ys_by_tag, cv_sent_td_predictions_by_tag, cv_td_preds_by_sent, \
    cv_sent_vd_ys_by_tag, cv_sent_vd_predictions_by_tag, cv_vd_preds_by_sent = result_test
    
# Predicted Concept Codes - correspond to Ana tags
preds_by_code_train = defaultdict(list)
ys_by_code_train    = defaultdict(list)

preds_by_code_test = defaultdict(list)
ys_by_code_test    = defaultdict(list)

for (orig_td, orig_vd), (td, vd), td_preds_by_sent, vd_preds_by_sent in zip(orig_test_folds, test_folds, cv_td_preds_by_sent, cv_vd_preds_by_sent):
    
    diffs_train = get_different_predicted_tags_by_sent(essays_a=orig_td, essays_b=td)
    diffs_test  = get_different_predicted_tags_by_sent(essays_a=orig_vd, essays_b=vd)

    # Actual Ana CRels
    diffs_crel_train = get_different_crel_tags_by_sent(essays_a=orig_td, essays_b=td)
    diffs_crel_test  = get_different_crel_tags_by_sent(essays_a=orig_vd, essays_b=vd)
    
    build_metrics_by_code(td_preds_by_sent, diffs_train, diffs_crel_train, ys_by_code_train, preds_by_code_train)
    build_metrics_by_code(vd_preds_by_sent, diffs_test,  diffs_crel_test,  ys_by_code_test,  preds_by_code_test)

## Test Metrics

In [27]:
df_train2 = get_metrics_from_dict(ys_by_code_train, preds_by_code_train, cr_tags)
df_train2[df_train2.code == "MICRO_F1"][["accuracy", "f1_score", "precision", "recall"]]

,accuracy,f1_score,precision,recall
95,0.999682,0.0625,0.5,0.033333


In [28]:
df_test2 = get_metrics_from_dict(ys_by_code_test, preds_by_code_test, cr_tags)
df_test2[df_test2.code == "MICRO_F1"][["accuracy", "f1_score", "precision", "recall"]]

,accuracy,f1_score,precision,recall
95,0.999794,0.0,0.0,0.0


In [35]:
[d for d in diffs_test if d]

[{'3'}, {'50'}, {'1'}]

In [36]:
# cv_vd_preds_by_sent

In [59]:
def get_different_predicted_tags_by_word(essays_a, essays_b):
    assert len(essays_a) == len(essays_b)
    diff_by_sent = defaultdict(list)
    i = -1
    for ea, eb in zip(essays_a, essays_b):
        assert len(ea.sentences) == len(eb.sentences)
        assert len(ea.pred_tagged_sentences) == len(eb.pred_tagged_sentences)
        for asent, bsent in zip(ea.pred_tagged_sentences, eb.pred_tagged_sentences):
            i += 1
            assert type(asent) == type(bsent)
            unique_atags = set(asent)
            if EMPTY in unique_atags:
                unique_atags.remove(EMPTY)
            unique_btags = set(bsent)
            if EMPTY in unique_btags:
                unique_btags.remove(EMPTY)
            diff_wd = unique_btags - unique_atags
            if diff_wd:
                diff_by_sent[i].append(diff_wd)
    return diff_by_sent

In [60]:
def get_different_predicted_tags_by_word2(essays_a, essays_b):
    assert len(essays_a) == len(essays_b)
    diff_by_sent = defaultdict(list)
    i = -1
    for ea, eb in zip(essays_a, essays_b):
        assert len(ea.sentences) == len(eb.sentences)
        assert len(ea.pred_tagged_sentences) == len(eb.pred_tagged_sentences)
        for asent, bsent in zip(ea.pred_tagged_sentences, eb.pred_tagged_sentences):
            i += 1
            assert type(asent) == type(bsent)        
            assert len(asent) == len(bsent)        
            for atag, btag in zip(asent, bsent):
                if atag != btag:
                    diff_by_sent[i].append(btag)
    return diff_by_sent

In [61]:
diff_sents = get_different_predicted_tags_by_word(orig_pred_tagged_essays_test, pred_tagged_essays_test)

In [62]:
diff_sents

defaultdict(list, {545: [{'3'}], 657: [{'50'}], 1229: [{'1'}]})

In [63]:
diff_sents2 = get_different_predicted_tags_by_word2(orig_pred_tagged_essays_test, pred_tagged_essays_test)

In [64]:
diff_sents2

defaultdict(list,
            {545: ['3'],
             657: ['50'],
             1110: ['14'],
             1112: ['13'],
             1229: ['1'],
             1835: ['5']})

In [47]:
count = 0
for sent_ix, diff in diff_sents.items():
    count += len(diff)
count

3

In [45]:
diff_by_sent = get_different_predicted_tags_by_sent(orig_pred_tagged_essays_test, pred_tagged_essays_test)
len([d for d in diff_by_sent if len(d) > 0])

3

In [55]:
diffs_crel_test  = get_different_crel_tags_by_sent(essays_a=orig_pred_tagged_essays_test, essays_b=pred_tagged_essays_test)

In [56]:
count = 0
for d in diffs_crel_test:
    if d:
        count += len(d)
count

33